In [1]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt

all_data = pd.read_csv('sales所在周次节假日.csv')
all_data.rename(columns={'店铺名称':'store','地理位置指数':'position','年份':'years','周次':'weeks of year','是否包含法定节假日':'Isholiday','季节':'season','大类':'item','销量':'sales','均价':'prices','促销':'discount'},inplace=True)
all_data['weeks'] = all_data['weeks of year']
a = all_data
week_5 = a.loc[lambda a : a['years'] == 2015]
week_6 = a.loc[lambda a : a['years'] == 2016]
week_6['weeks'] += 53
a = pd.concat([week_5,week_6])

from sklearn import preprocessing

#map店铺名称
name = a['store']
le = preprocessing.LabelEncoder()
le.fit(name)
list(name)
le.transform(name)
tmp = le.transform(name)
a['store'] = tmp

#map合并大类
category = a['item']
le.fit(category)
list(category)
le.transform(category)
tmp = le.transform(category)
a['item'] = tmp

#map季节
season = a['season']
le.fit(season)
list(season)
le.transform(season)
tmp = le.transform(season)
a['season'] = tmp

a

/home/lin/anaconda3/envs/boost/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


,store,position,years,weeks of year,Isholiday,season,item,sales,prices,discount,weeks
0,0,7,2015,1,1,0,1,1,499.666700,0.333333,1
1,0,7,2015,1,1,0,2,30,513.066663,0.371160,1
2,0,7,2015,1,1,0,9,2,683.833300,0.325790,1
3,0,7,2015,1,1,0,10,2,799.600000,0.400000,1
4,0,7,2015,1,1,2,6,1,375.600000,0.400000,1
5,0,7,2015,1,1,3,7,2,366.333350,0.333333,1
6,0,7,2015,2,0,0,1,1,799.000000,0.470277,2
7,0,7,2015,2,0,0,2,6,645.833333,0.467205,2
8,0,7,2015,2,0,2,6,1,563.000000,0.599574,2
9,0,7,2015,2,0,3,6,2,354.000000,0.385201,2


In [2]:
df = a
masked_series = (df['years']==2016) & (df['weeks'].isin([53,54,55]))
masked_series2 = (df['years']==2016) & (~(df['weeks'].isin([53,54,55])))
df.loc[(masked_series), 'train_or_test'] = 'test'
df.loc[(masked_series2), 'train_or_test'] = 'no_train'
print('Train shape: {}'.format(df.loc[df['years']==2015,:].shape))
print('Test shape: {}'.format(df.loc[df.train_or_test=='test',:].shape))


Train shape: (8119, 12)
Test shape: (340, 12)


In [3]:
df_n = df.drop(['train_or_test'],axis=1)

#log sales,prices
#df_n['sales'] = np.log1p(df.sales.values)
df_n['prices'] = np.log1p(df.sales.values)
df_n.sample(2)
#df_n = df_n.drop(['prices','discount'],axis=1)

,store,position,years,weeks of year,Isholiday,season,item,sales,prices,discount,weeks
5980,9,7,2016,11,0,2,7,1,0.693147,0.276897,64
10135,16,7,2016,48,0,0,2,38,3.663562,0.258857,101


In [4]:
train = df_n[lambda df_n: df_n['weeks'] < 54]
#train = df_n.sort_values('weeks',ascending='True')
#train = train.drop(['prices','discount'],axis=1)
y_train = train['sales']

In [5]:
#train = df_n[lambda df_n: df_n['years']==2015]
#t1 = df_n[lambda df_n: df_n['weeks']==54]
#t2 = df_n[lambda df_n: df_n['weeks']==55]
#t3 = df_n[lambda df_n: df_n['weeks']==56]
#test = pd.concat([t1,t2,t3])

#y_train = train['sales']
#y_test = test['sales']
#X_train = train.drop('sales', axis=1)
#X_test = test.drop('sales', axis=1)

In [6]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn import svm
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error,r2_score
import xgboost as xgb
import lightgbm as lgb

In [7]:
#Validation function
n_folds = 3

def rmsle_cv(model):
    kf = KFold(n_folds).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [8]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
model_svm = svm.SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
                    gamma='auto', kernel='rbf', max_iter=-1, shrinking=True,
                    tol=0.001, verbose=False)

model_regr = RandomForestRegressor(max_depth=2, random_state=0,
                                    n_estimators=100)
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
model_lgb = lgb.LGBMRegressor(n_jobs = -1)
model_mlp = MLPRegressor(activation='identity')


In [9]:
# score = rmsle_cv(lasso)
# print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
# score = rmsle_cv(ENet)
# print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
# #score = rmsle_cv(KRR)
# #print("Kernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
# #score = rmsle_cv(GBoost)
# #print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
# score = rmsle_cv(model_xgb)
# print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
# score = rmsle_cv(model_lgb)
# print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

In [10]:
df_test = df_n[lambda df_n: df_n['weeks'] > 53]
df_test_1 = df_test[lambda df_n: df_n['weeks'] <67]
df_test_2_ = df_test[lambda df_n: df_n['weeks'] <79]
df_test_2 = df_test_2_[lambda df_n: df_n['weeks'] >67]
df_test_3_ = df_test[lambda df_n: df_n['weeks'] <92]
df_test_3 = df_test_3_[lambda df_n: df_n['weeks'] >79]
df_test_4 = df_test[lambda df_n: df_n['weeks'] >92]

In [11]:
df_train = df_n[lambda df_n: df_n['weeks'] < 53]
df_train_1 = df_train[lambda df_n: df_n['weeks'] <14]
df_train_2_ = df_train[lambda df_n: df_n['weeks'] <27]
df_train_2 = df_train_2_[lambda df_n: df_n['weeks'] >14]
df_train_3_ = df_train[lambda df_n: df_n['weeks'] <40]
df_train_3 = df_train_3_[lambda df_n: df_n['weeks'] >27]
df_train_4 = df_train[lambda df_n: df_n['weeks'] >40]

In [12]:
df_test_0_x = df_test.drop(['sales'],axis=1)
df_test_0_y = df_test['sales']
df_test_1_x = df_test_1.drop(['sales'],axis=1)
df_test_1_y = df_test_1['sales']
df_test_2_x = df_test_2.drop(['sales'],axis=1)
df_test_2_y = df_test_2['sales']
df_test_3_x = df_test_3.drop(['sales'],axis=1)
df_test_3_y = df_test_3['sales']
df_test_4_x = df_test_4.drop(['sales'],axis=1)
df_test_4_y = df_test_4['sales']

df_train_0_x = df_train.drop(['sales'],axis=1)
df_train_0_y = df_train['sales']
df_train_1_x = df_train_1.drop(['sales'],axis=1)
df_train_1_y = df_train_1['sales']
df_train_2_x = df_train_2.drop(['sales'],axis=1)
df_train_2_y = df_train_2['sales']
df_train_3_x = df_train_3.drop(['sales'],axis=1)
df_train_3_y = df_train_3['sales']
df_train_4_x = df_train_4.drop(['sales'],axis=1)
df_train_4_y = df_train_4['sales']

In [13]:
def rmse_test(model):
    model = model.fit(df_train_0_x,df_train_0_y)
    y_pred_1 = model.predict(df_test_1_x)
    print('The rmse of prediction is:', mean_squared_error(df_test_1_y, y_pred_1) ** 0.5)
    print('The R^2 of prediction is:', r2_score(df_test_1_y, y_pred_1))


In [14]:
rmse_test(lasso)
rmse_test(model_svm)
rmse_test(model_regr)
rmse_test(GBoost)
rmse_test(model_xgb)
rmse_test(model_lgb)
rmse_test(model_mlp)


The rmse of prediction is: 9.285444513793408
The R^2 of prediction is: 0.5923771291385977
The rmse of prediction is: 14.94038700656131
The R^2 of prediction is: -0.05530219066943398
The rmse of prediction is: 6.946444143704363
The R^2 of prediction is: 0.7718721849972066
The rmse of prediction is: 1.890782373026477
The R^2 of prediction is: 0.98309807248614
The rmse of prediction is: 0.47776195956283984
The R^2 of prediction is: 0.9989208637544054
The rmse of prediction is: 7.482878370862198
The R^2 of prediction is: 0.7352777062177414
The rmse of prediction is: 12.96768885270675
The R^2 of prediction is: 0.20497953532074786


# 第一次迭代

In [15]:
#去头，尾变
model_xgb.fit(df_train_0_x,df_train_0_y)
y_pred_1 = pd.DataFrame(model_xgb.predict(df_test_1_x))

In [16]:
train_ite_1_x = pd.concat([df_train_2_x,df_train_3_x,df_train_4_x,df_test_1_x])
train_ite_1_y = pd.concat([df_train_2_y,df_train_3_y,df_train_4_y,y_pred_1])

In [17]:
def rmse_test(model):
    model = model.fit(train_ite_1_x,train_ite_1_y)
    y_pred_2 = model.predict(df_test_2_x)
    print('The rmse of prediction is:', mean_squared_error(df_test_2_y, y_pred_2) ** 0.5)
    print('The R^2 of prediction is:', r2_score(df_test_2_y, y_pred_2))

rmse_test(lasso)
rmse_test(model_svm)
rmse_test(model_regr)
rmse_test(GBoost)
rmse_test(model_xgb)
rmse_test(model_lgb)
rmse_test(model_mlp)


The rmse of prediction is: 5.647801847532188
The R^2 of prediction is: 0.6520236063511134


/home/lin/anaconda3/envs/boost/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The rmse of prediction is: 9.93614735906504
The R^2 of prediction is: -0.0770278364352508


/home/lin/anaconda3/envs/boost/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


The rmse of prediction is: 3.61677288712439
The R^2 of prediction is: 0.8572969165051136


/home/lin/anaconda3/envs/boost/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The rmse of prediction is: 0.46036309915229795
The R^2 of prediction is: 0.9976879783810717
The rmse of prediction is: 0.8631597239585626
The R^2 of prediction is: 0.991872196124549
The rmse of prediction is: 0.2189577423466082
The R^2 of prediction is: 0.9994769881926236


/home/lin/anaconda3/envs/boost/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:1316: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The rmse of prediction is: 6.794968464751914
The R^2 of prediction is: 0.49630716668328323


In [18]:
#去头，尾不变
train_ite_1_x = pd.concat([df_train_2_x,df_train_3_x,df_train_4_x,df_test_1_x])
train_ite_1_y = pd.concat([df_train_2_y,df_train_3_y,df_train_4_y,df_test_1_y])

def rmse_test(model):
    model = model.fit(train_ite_1_x,train_ite_1_y)
    y_pred_2 = model.predict(df_test_2_x)
    print('The rmse of prediction is:', mean_squared_error(df_test_2_y, y_pred_2) ** 0.5)
    print('The R^2 of prediction is:', r2_score(df_test_2_y, y_pred_2))

rmse_test(lasso)
rmse_test(model_svm)
rmse_test(model_regr)
rmse_test(GBoost)
rmse_test(model_xgb)
rmse_test(model_lgb)
rmse_test(model_mlp)


The rmse of prediction is: 5.617681422609116
The R^2 of prediction is: 0.6557253113200043
The rmse of prediction is: 9.94635208076817
The R^2 of prediction is: -0.07924125233278856
The rmse of prediction is: 3.628724605569758
The R^2 of prediction is: 0.8563522262298413
The rmse of prediction is: 0.3443499433985121
The R^2 of prediction is: 0.9987064270886353
The rmse of prediction is: 0.8386044408829846
The R^2 of prediction is: 0.9923280600226732
The rmse of prediction is: 0.10092962466131841
The R^2 of prediction is: 0.9998888707976895
The rmse of prediction is: 6.729441935164883
The R^2 of prediction is: 0.5059749384476326


In [19]:
#不去头，尾变
train_ite_1_x = pd.concat([df_train_1_x,df_train_2_x,df_train_3_x,df_train_4_x,df_test_1_x])
train_ite_1_y = pd.concat([df_train_1_y,df_train_2_y,df_train_3_y,df_train_4_y,y_pred_1])

def rmse_test(model):
    model = model.fit(train_ite_1_x,train_ite_1_y)
    y_pred_2 = model.predict(df_test_2_x)
    print('The rmse of prediction is:', mean_squared_error(df_test_2_y, y_pred_2) ** 0.5)
    print('The R^2 of prediction is:', r2_score(df_test_2_y, y_pred_2))

rmse_test(lasso)
rmse_test(model_svm)
rmse_test(model_regr)
rmse_test(GBoost)
rmse_test(model_xgb)
rmse_test(model_lgb)
rmse_test(model_mlp)


The rmse of prediction is: 5.925597220373844
The R^2 of prediction is: 0.6169502912786842


/home/lin/anaconda3/envs/boost/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The rmse of prediction is: 9.935000846104202
The R^2 of prediction is: -0.07677929842813103


/home/lin/anaconda3/envs/boost/lib/python3.7/site-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


The rmse of prediction is: 4.610590641634517
The R^2 of prediction is: 0.7680982588219699


/home/lin/anaconda3/envs/boost/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The rmse of prediction is: 0.43582480144939106
The R^2 of prediction is: 0.9979278806679427
The rmse of prediction is: 0.3423496734122777
The R^2 of prediction is: 0.9987214117283061
The rmse of prediction is: 0.251275785086611
The R^2 of prediction is: 0.9993112015554293


/home/lin/anaconda3/envs/boost/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:1316: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The rmse of prediction is: 5.645144560776725
The R^2 of prediction is: 0.6523509746115462


In [20]:
#不去头，尾不变
train_ite_1_x = pd.concat([df_train_1_x,df_train_2_x,df_train_3_x,df_train_4_x,df_test_1_x])
train_ite_1_y = pd.concat([df_train_1_y,df_train_2_y,df_train_3_y,df_train_4_y,df_test_1_y])

def rmse_test(model):
    model = model.fit(train_ite_1_x,train_ite_1_y)
    y_pred_2 = model.predict(df_test_2_x)
    print('The rmse of prediction is:', mean_squared_error(df_test_2_y, y_pred_2) ** 0.5)
    print('The R^2 of prediction is:', r2_score(df_test_2_y, y_pred_2))

rmse_test(lasso)
rmse_test(model_svm)
rmse_test(model_regr)
rmse_test(GBoost)
rmse_test(model_xgb)
rmse_test(model_lgb)
rmse_test(model_mlp)


The rmse of prediction is: 5.9114348263965715
The R^2 of prediction is: 0.6187791088073011
The rmse of prediction is: 9.942589070972495
The R^2 of prediction is: -0.07842478673130682
The rmse of prediction is: 4.647315355372115
The R^2 of prediction is: 0.7643892141285383
The rmse of prediction is: 0.33858697975490676
The R^2 of prediction is: 0.9987493626706103
The rmse of prediction is: 0.30249766040691445
The R^2 of prediction is: 0.999001760109369
The rmse of prediction is: 0.18372201550835293
The R^2 of prediction is: 0.9996317749897596
The rmse of prediction is: 5.563687182506325
The R^2 of prediction is: 0.6623114865111179


# 第二次迭代

In [21]:
#去头，尾变
model_xgb.fit(train_ite_1_x,train_ite_1_y)
y_pred_2 = pd.DataFrame(model_xgb.predict(df_test_2_x))

train_ite_2_x = pd.concat([df_train_3_x,df_train_4_x,df_test_1_x,df_test_2_x])
train_ite_2_y = pd.concat([df_train_3_y,df_train_4_y,df_test_1_y,y_pred_2])

def rmse_test(model):
    model = model.fit(train_ite_2_x,train_ite_2_y)
    y_pred_3 = model.predict(df_test_3_x)
    print('The rmse of prediction is:', mean_squared_error(df_test_3_y, y_pred_3) ** 0.5)
    print('The R^2 of prediction is:', r2_score(df_test_3_y, y_pred_3))
    
rmse_test(lasso)
rmse_test(model_svm)
rmse_test(model_regr)
rmse_test(GBoost)
rmse_test(model_xgb)
rmse_test(model_lgb)
rmse_test(model_mlp)


The rmse of prediction is: 6.707917781920192
The R^2 of prediction is: 0.532921135233014


/home/lin/anaconda3/envs/boost/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The rmse of prediction is: 9.925077399503191
The R^2 of prediction is: -0.02254530432827817


/home/lin/anaconda3/envs/boost/lib/python3.7/site-packages/ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':


The rmse of prediction is: 3.275257984887651
The R^2 of prediction is: 0.8886460101830775


/home/lin/anaconda3/envs/boost/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The rmse of prediction is: 2.1474677042325494
The R^2 of prediction is: 0.9521294919550182
The rmse of prediction is: 0.583506138108532
The R^2 of prediction is: 0.9964656820854478
The rmse of prediction is: 2.055654110000305
The R^2 of prediction is: 0.9561353330739177


/home/lin/anaconda3/envs/boost/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:1316: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The rmse of prediction is: 6.544755309075126
The R^2 of prediction is: 0.5553671124704354


In [22]:
#去头，尾不变
train_ite_2_x = pd.concat([df_train_3_x,df_train_4_x,df_test_1_x,df_test_2_x])
train_ite_2_y = pd.concat([df_train_3_y,df_train_4_y,df_test_1_y,df_test_2_y])

def rmse_test(model):
    model = model.fit(train_ite_2_x,train_ite_2_y)
    y_pred_3 = model.predict(df_test_3_x)
    print('The rmse of prediction is:', mean_squared_error(df_test_3_y, y_pred_3) ** 0.5)
    print('The R^2 of prediction is:', r2_score(df_test_3_y, y_pred_3))
    
rmse_test(lasso)
rmse_test(model_svm)
rmse_test(model_regr)
rmse_test(GBoost)
rmse_test(model_xgb)
rmse_test(model_lgb)
rmse_test(model_mlp)


The rmse of prediction is: 6.714862708566507
The R^2 of prediction is: 0.5319534705179101
The rmse of prediction is: 9.92409784435741
The R^2 of prediction is: -0.022343474146820475
The rmse of prediction is: 3.275973825220985
The R^2 of prediction is: 0.8885973298113645
The rmse of prediction is: 2.2641915057678728
The R^2 of prediction is: 0.9467841421074249
The rmse of prediction is: 0.5458709309224247
The R^2 of prediction is: 0.9969068948120442
The rmse of prediction is: 2.0501972741177408
The R^2 of prediction is: 0.9563679058474768
The rmse of prediction is: 7.516557597387586
The R^2 of prediction is: 0.4135206385775265


In [23]:
#不去头，尾变
model_xgb.fit(train_ite_1_x,train_ite_1_y)
y_pred_2 = pd.DataFrame(model_xgb.predict(df_test_2_x))

train_ite_2_x = pd.concat([df_train_1_x,df_train_2_x,df_train_3_x,df_train_4_x,df_test_1_x,df_test_2_x])
train_ite_2_y = pd.concat([df_train_1_y,df_train_2_y,df_train_3_y,df_train_4_y,df_test_1_y,y_pred_2])

def rmse_test(model):
    model = model.fit(train_ite_2_x,train_ite_2_y)
    y_pred_3 = model.predict(df_test_3_x)
    print('The rmse of prediction is:', mean_squared_error(df_test_3_y, y_pred_3) ** 0.5)
    print('The R^2 of prediction is:', r2_score(df_test_3_y, y_pred_3))
    
rmse_test(lasso)
rmse_test(model_svm)
rmse_test(model_regr)
rmse_test(GBoost)
rmse_test(model_xgb)
rmse_test(model_lgb)
rmse_test(model_mlp)


The rmse of prediction is: 6.924072751320011
The R^2 of prediction is: 0.5023339682179548


/home/lin/anaconda3/envs/boost/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The rmse of prediction is: 9.91355079587158
The R^2 of prediction is: -0.020171593824089173


/home/lin/anaconda3/envs/boost/lib/python3.7/site-packages/ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':


The rmse of prediction is: 5.355235390805435
The R^2 of prediction is: 0.7023047641653286


/home/lin/anaconda3/envs/boost/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The rmse of prediction is: 0.9843020213580695
The R^2 of prediction is: 0.9899429291841627
The rmse of prediction is: 0.39084798962453215
The R^2 of prediction is: 0.9984142653388901
The rmse of prediction is: 3.000873786722742
The R^2 of prediction is: 0.9065218273558968


/home/lin/anaconda3/envs/boost/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:1316: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The rmse of prediction is: 7.888643156063463
The R^2 of prediction is: 0.35401954729052165


In [24]:
#不去头，尾不变
model_xgb.fit(train_ite_1_x,train_ite_1_y)
y_pred_2 = pd.DataFrame(model_xgb.predict(df_test_2_x))

train_ite_2_x = pd.concat([df_train_1_x,df_train_2_x,df_train_3_x,df_train_4_x,df_test_1_x,df_test_2_x])
train_ite_2_y = pd.concat([df_train_1_y,df_train_2_y,df_train_3_y,df_train_4_y,df_test_1_y,df_test_2_y])

def rmse_test(model):
    model = model.fit(train_ite_2_x,train_ite_2_y)
    y_pred_3 = model.predict(df_test_3_x)
    print('The rmse of prediction is:', mean_squared_error(df_test_3_y, y_pred_3) ** 0.5)
    print('The R^2 of prediction is:', r2_score(df_test_3_y, y_pred_3))
    
rmse_test(lasso)
rmse_test(model_svm)
rmse_test(model_regr)
rmse_test(GBoost)
rmse_test(model_xgb)
rmse_test(model_lgb)
rmse_test(model_mlp)


The rmse of prediction is: 6.929617475123704
The R^2 of prediction is: 0.5015365977542914
The rmse of prediction is: 9.912656634077067
The R^2 of prediction is: -0.019987571500798884
The rmse of prediction is: 5.339747391810676
The R^2 of prediction is: 0.7040242165578832
The rmse of prediction is: 0.8831240927628847
The R^2 of prediction is: 0.9919042289018956
The rmse of prediction is: 0.3620684668012812
The R^2 of prediction is: 0.9986391941732805
The rmse of prediction is: 3.075221848131007
The R^2 of prediction is: 0.9018325167069583
The rmse of prediction is: 7.016147776561435
The R^2 of prediction is: 0.48901022528154603


# 第三次迭代

In [25]:
#去头，尾变
model_xgb.fit(train_ite_2_x,train_ite_2_y)
y_pred_3 = pd.DataFrame(model_xgb.predict(df_test_3_x))

train_ite_3_x = pd.concat([df_train_4_x,df_test_1_x,df_test_2_x,df_test_3_x])
train_ite_3_y = pd.concat([df_train_4_y,df_test_1_y,df_test_2_y,y_pred_3])

def rmse_test(model):
    model = model.fit(train_ite_3_x,train_ite_3_y)
    y_pred_4 = model.predict(df_test_4_x)
    print('The rmse of prediction is:', mean_squared_error(df_test_4_y, y_pred_4) ** 0.5)
    print('The R^2 of prediction is:', r2_score(df_test_4_y, y_pred_4))
    
rmse_test(lasso)
rmse_test(model_svm)
rmse_test(model_regr)
rmse_test(GBoost)
rmse_test(model_xgb)
rmse_test(model_lgb)
rmse_test(model_mlp)


The rmse of prediction is: 9.591736603023953
The R^2 of prediction is: 0.5808375592739167


/home/lin/anaconda3/envs/boost/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The rmse of prediction is: 15.41425623388252
The R^2 of prediction is: -0.08251275018265503


/home/lin/anaconda3/envs/boost/lib/python3.7/site-packages/ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':


The rmse of prediction is: 4.08184779022722
The R^2 of prediction is: 0.9240895642595617


/home/lin/anaconda3/envs/boost/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The rmse of prediction is: 3.8431966096049317
The R^2 of prediction is: 0.9327065061334872
The rmse of prediction is: 1.0467077582536761
The R^2 of prediction is: 0.9950084116943271
The rmse of prediction is: 2.647313868158487
The R^2 of prediction is: 0.9680700130040083


/home/lin/anaconda3/envs/boost/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:1316: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The rmse of prediction is: 9.807352736032822
The R^2 of prediction is: 0.5617807376819643


In [26]:
#去头，尾不变
train_ite_3_x = pd.concat([df_train_4_x,df_test_1_x,df_test_2_x,df_test_3_x])
train_ite_3_y = pd.concat([df_train_4_y,df_test_1_y,df_test_2_y,df_test_3_y])

def rmse_test(model):
    model = model.fit(train_ite_3_x,train_ite_3_y)
    y_pred_4 = model.predict(df_test_4_x)
    print('The rmse of prediction is:', mean_squared_error(df_test_4_y, y_pred_4) ** 0.5)
    print('The R^2 of prediction is:', r2_score(df_test_4_y, y_pred_4))
    
rmse_test(lasso)
rmse_test(model_svm)
rmse_test(model_regr)
rmse_test(GBoost)
rmse_test(model_xgb)
rmse_test(model_lgb)
rmse_test(model_mlp)


The rmse of prediction is: 9.591281273900368
The R^2 of prediction is: 0.5808773544318935
The rmse of prediction is: 15.412286038789821
The R^2 of prediction is: -0.0822360420528725
The rmse of prediction is: 4.1018814319813846
The R^2 of prediction is: 0.9233426013682595
The rmse of prediction is: 3.7439717611261125
The R^2 of prediction is: 0.9361364581018302
The rmse of prediction is: 0.9996616033824921
The R^2 of prediction is: 0.9954470393705185
The rmse of prediction is: 2.650406384076976
The R^2 of prediction is: 0.9679953700467097
The rmse of prediction is: 10.406520172868257
The R^2 of prediction is: 0.5066002409584656


In [27]:
#不去头，尾变
model_xgb.fit(train_ite_2_x,train_ite_2_y)
y_pred_3 = pd.DataFrame(model_xgb.predict(df_test_3_x))

train_ite_3_x = pd.concat([df_train_1_x,df_train_2_x,df_train_3_x,df_train_4_x,df_test_1_x,df_test_2_x,df_test_3_x])
train_ite_3_y = pd.concat([df_train_1_y,df_train_2_y,df_train_3_y,df_train_4_y,df_test_1_y,df_test_2_y,y_pred_3])

def rmse_test(model):
    model = model.fit(train_ite_3_x,train_ite_3_y)
    y_pred_4 = model.predict(df_test_4_x)
    print('The rmse of prediction is:', mean_squared_error(df_test_4_y, y_pred_4) ** 0.5)
    print('The R^2 of prediction is:', r2_score(df_test_4_y, y_pred_4))
    
rmse_test(lasso)
rmse_test(model_svm)
rmse_test(model_regr)
rmse_test(GBoost)
rmse_test(model_xgb)
rmse_test(model_lgb)
rmse_test(model_mlp)


The rmse of prediction is: 9.510569710749833
The R^2 of prediction is: 0.5879015902512397


/home/lin/anaconda3/envs/boost/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The rmse of prediction is: 15.430919955649333
The R^2 of prediction is: -0.08485453579329749


/home/lin/anaconda3/envs/boost/lib/python3.7/site-packages/ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':


The rmse of prediction is: 6.799254507289281
The R^2 of prediction is: 0.7893747334319423


/home/lin/anaconda3/envs/boost/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The rmse of prediction is: 2.0746253354884248
The R^2 of prediction is: 0.9803904681035739
The rmse of prediction is: 0.42811028918357225
The R^2 of prediction is: 0.9991649755275568
The rmse of prediction is: 5.2846467131063815
The R^2 of prediction is: 0.8727611336989429


/home/lin/anaconda3/envs/boost/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:1316: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The rmse of prediction is: 9.606530141997883
The R^2 of prediction is: 0.5795435959288382


In [28]:
#不去头，尾不变
model_xgb.fit(train_ite_2_x,train_ite_2_y)
y_pred_3 = pd.DataFrame(model_xgb.predict(df_test_3_x))

train_ite_3_x = pd.concat([df_train_1_x,df_train_2_x,df_train_3_x,df_train_4_x,df_test_1_x,df_test_2_x,df_test_3_x])
train_ite_3_y = pd.concat([df_train_1_y,df_train_2_y,df_train_3_y,df_train_4_y,df_test_1_y,df_test_2_y,df_test_3_y])

def rmse_test(model):
    model = model.fit(train_ite_3_x,train_ite_3_y)
    y_pred_4 = model.predict(df_test_4_x)
    print('The rmse of prediction is:', mean_squared_error(df_test_4_y, y_pred_4) ** 0.5)
    print('The R^2 of prediction is:', r2_score(df_test_4_y, y_pred_4))
    
rmse_test(lasso)
rmse_test(model_svm)
rmse_test(model_regr)
rmse_test(GBoost)
rmse_test(model_xgb)
rmse_test(model_lgb)
rmse_test(model_mlp)


The rmse of prediction is: 9.51034406461958
The R^2 of prediction is: 0.5879211447702972
The rmse of prediction is: 15.429116741892118
The R^2 of prediction is: -0.08460100452883679
The rmse of prediction is: 6.862688147089762
The R^2 of prediction is: 0.785426344170864
The rmse of prediction is: 1.9652897432127945
The R^2 of prediction is: 0.9824029022150498
The rmse of prediction is: 0.5683778869936248
The R^2 of prediction is: 0.9985281545199678
The rmse of prediction is: 5.254221602192258
The R^2 of prediction is: 0.8742220119390486
The rmse of prediction is: 10.399627879047493
The R^2 of prediction is: 0.5072535871140671
